In [29]:
import numpy as np
import torch
import torch.nn.functional as F
from processor import (VLSP_2018_QA_M_Processor, VLSP_2018_QA_B_Processor, VLSP_2018_NLI_M_Processor, VLSP_2018_NLI_B_Processor)

In [30]:
torch.manual_seed(42)

In [31]:
import tokenization
from modeling import BertConfig, BertForSequenceClassification

In [32]:
bert_config = BertConfig.from_json_file('/Users/vinhtruongtrong/Downloads/drive-download-20201015T092207Z-001/bert_config.json')

In [33]:
processors = {
        "vlsp_2018_NLI_M":VLSP_2018_NLI_M_Processor,
        "vlsp_2018_QA_M":VLSP_2018_QA_M_Processor,
        "vlsp_2018_NLI_B":VLSP_2018_NLI_B_Processor,
        "vlsp_2018_QA_B":VLSP_2018_QA_B_Processor,
    }
processor = processors['vlsp_2018_QA_M']()
label_list = processor.get_labels()

tokenizer = tokenization.FullTokenizer(
    vocab_file='/Users/vinhtruongtrong/Downloads/drive-download-20201015T092207Z-001/vocab.txt', do_lower_case=False)

# Sample generator

In [5]:
from text_pre_processing.text_pre_processing import TextPreProcessing

In [6]:
text_processing = TextPreProcessing()

In [7]:
sentence = 'Nay được thằng bạn rủ đi ăn mà cũng lâu rồi mình không ăn quán này, nay ghé lại vẫn đông và ổn định như ngày nào. Sẵn viết nhẹ bài review luôn nà.  Hột vịt lộn chiên nước mắm  Há cảo hấp  Há cảo chiên trứng  Bánh flan  Phá lấu  Hàu nướng phô mai   Cứ mỗi lần là đều double lên hết nha mấy chế. Ăn ở đây nên gọi full bàn vì làm sẽ rất lâu. Bù lại giá cả mềm mại, đồ ăn làm cũng ok lắm.'

In [9]:
sentence = text_processing.sentence_pre_processing(sentence)

In [10]:
print(sentence)

Nay được thằng bạn rủ đi ăn mà cũng lâu rồi mình không ăn quán này nay ghé lại vẫn đông và ổn_định như ngày nào . Sẵn viết nhẹ bài review luôn nà . Hột_vịt_lộn chiên nước_mắm Há cảo hấp Há cảo chiên trứng Bánh flan Phá lấu Hàu nướng phô mai Cứ mỗi lần là đều double lên hết nha mấy chế . Ăn ở đây nên gọi ful bàn vì làm sẽ rất lâu . Bù_lại giá_cả mềm_mại đồ ăn làm cũng ok lắm .


In [11]:
from text_pre_processing.method.qam import QA_M
def generate_QA_M(val, domain):
        result = []
        for aspect in QA_M(domain).aspect_categories:
            question = 'bạn nghĩ thế_nào về '+ QA_M(domain).aspect_categories[aspect] + ' ?'
            result.append([question, val])
        return result

In [15]:
qa_m_sentences = generate_QA_M(sentence, 'restaurant')

In [17]:
qa_m_sentences[0]

['bạn nghĩ thế_nào về phong_cách đồ_ăn ?',
 'Nay được thằng bạn rủ đi ăn mà cũng lâu rồi mình không ăn quán này nay ghé lại vẫn đông và ổn_định như ngày nào . Sẵn viết nhẹ bài review luôn nà . Hột_vịt_lộn chiên nước_mắm Há cảo hấp Há cảo chiên trứng Bánh flan Phá lấu Hàu nướng phô mai Cứ mỗi lần là đều double lên hết nha mấy chế . Ăn ở đây nên gọi ful bàn vì làm sẽ rất lâu . Bù_lại giá_cả mềm_mại đồ ăn làm cũng ok lắm .']

In [20]:
from processor import InputExample
import tokenization
def create_examples(lines, set_type = 'test'):
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = tokenization.convert_to_unicode(str(line[1]))
            text_b = tokenization.convert_to_unicode(str(line[0]))
            if i%1000==0:
                print(i)
                print("guid=",guid)
                print("text_a=",text_a)
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=text_b))
        return examples

In [27]:
test_examples = create_examples(qa_m_sentences)

0
guid= test-0
text_a= Nay được thằng bạn rủ đi ăn mà cũng lâu rồi mình không ăn quán này nay ghé lại vẫn đông và ổn_định như ngày nào . Sẵn viết nhẹ bài review luôn nà . Hột_vịt_lộn chiên nước_mắm Há cảo hấp Há cảo chiên trứng Bánh flan Phá lấu Hàu nướng phô mai Cứ mỗi lần là đều double lên hết nha mấy chế . Ăn ở đây nên gọi ful bàn vì làm sẽ rất lâu . Bù_lại giá_cả mềm_mại đồ ăn làm cũng ok lắm .


In [23]:
class InputFeatures(object):
    def __init__(self, input_ids, input_mask, segment_ids):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids

In [25]:
from tqdm import tqdm, trange
def convert_examples_to_features(examples, max_seq_length, tokenizer):
    
    features = []
    for (ex_index, example) in enumerate(tqdm(examples)):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)

        if tokens_b:
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)]

        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        input_mask = [1] * len(input_ids)
        
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        features.append(
                InputFeatures(
                        input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids))
    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [34]:
test_features = convert_examples_to_features(test_examples, 128, tokenizer)

100%|██████████| 12/12 [00:00<00:00, 616.90it/s]


In [56]:
import run_classifier_TABSA
all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

test_data = run_classifier_TABSA.TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
test_dataloader = run_classifier_TABSA.DataLoader(test_data,batch_size=len(test_features), shuffle=False)

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
model = BertForSequenceClassification(bert_config, len(label_list))
model.load_state_dict(torch.load('/Users/vinhtruongtrong/Downloads/drive-download-20201015T092207Z-001/model.bin', map_location='cpu'))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BERTEmbeddings(
      (word_embeddings): Embedding(119547, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BERTLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BERTEncoder(
      (layer): ModuleList(
        (0): BERTLayer(
          (attention): BERTAttention(
            (self): BERTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BERTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BERTLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
   

In [58]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BERTEmbeddings(
      (word_embeddings): Embedding(119547, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BERTLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BERTEncoder(
      (layer): ModuleList(
        (0): BERTLayer(
          (attention): BERTAttention(
            (self): BERTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BERTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BERTLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
   

In [59]:
for input_ids, input_mask, segment_ids in test_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask)

    logits = F.softmax(logits, dim=-1)
    logits = logits.detach().cpu().numpy()
    outputs = np.argmax(logits, axis=1)
    
    print(outputs)

[3 0 3 3 3 3 3 3 3 3 3 0]


In [61]:
QA_M('restaurant').aspect_categories

{'FOOD#STYLE&OPTIONS': 'phong_cách đồ_ăn',
 'FOOD#QUALITY': 'chất_lượng đồ_ăn',
 'AMBIENCE#GENERAL': 'cảnh_quan',
 'RESTAURANT#GENERAL': 'tổng_thể',
 'SERVICE#GENERAL': 'dịch_vụ',
 'FOOD#PRICES': 'giá thức_ăn',
 'RESTAURANT#PRICES': 'giá tổng_thể',
 'LOCATION#GENERAL': 'vị_trí',
 'RESTAURANT#MISCELLANEOUS': 'khác',
 'DRINKS#STYLE&OPTIONS': 'phong_cách nước',
 'DRINKS#PRICES': 'giá nước',
 'DRINKS#QUALITY': 'chất_lượng nước'}

In [66]:
def create_sample(sentence, domain = 'restaurant', method = 'QA_M'):
    text_processing = TextPreProcessing()
    sentence = text_processing.sentence_pre_processing(sentence)
    auxilary_sentences = generate_QA_M(sentence, domain) if method == 'QA_M' else None
    
    test_examples = create_examples(auxilary_sentences)
    test_features = convert_examples_to_features(test_examples, 128, tokenizer)
    
    all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

    test_data = run_classifier_TABSA.TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    test_dataloader = run_classifier_TABSA.DataLoader(test_data,batch_size=len(test_features), shuffle=False)
    
    return test_dataloader

In [67]:
def predict(sentence, domain = 'restaurant', method = 'QA_M'): 
    test_dataloader = create_sample(sentence, domain, method)
    model.eval()
    ouputs = None
    for input_ids, input_mask, segment_ids in test_dataloader:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)

        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask)

        logits = F.softmax(logits, dim=-1)
        logits = logits.detach().cpu().numpy()
        outputs = np.argmax(logits, axis=1)
    
    return outputs

In [75]:
label_list

['tích_cực', 'trung_lập', 'tiêu_cực', 'không_có']

In [74]:
label_map

{'tích_cực': 0, 'trung_lập': 1, 'tiêu_cực': 2, 'không_có': 3}

In [83]:
def multiple_output_post_processing(outputs, aspect_categories, label_list):
    result = []
    for (index, value) in enumerate(aspect_categories):
        if(outputs[index] != 3):
            result.append([aspect_categories[value], label_list[outputs[index]]])
    return result

In [86]:
sentence = 'vào đây 1 lần rồi thích mê chỗ này vì phong cách phục vụ kiểu alice đúng như tên wuan. Đồ ăn đến thức uống đều ngon giá hợp túi tiền sanh viên.thich nhất mấy món kem và soda, đồ ăn món nào cũng ok  mì spagetti  2 ly cream ngon thần thánh của mềnh   cơm gà chiên giòn  cơm cuộn ăn đỡ ngán vì có sốt cà chua, gà bên trong hơi ít'

In [87]:
r = predict(sentence)

100%|██████████| 12/12 [00:00<00:00, 265.65it/s]

0
guid= test-0
text_a= vào đây lần rồi thích mê chỗ này vì phong_cách phục_vụ kiểu alice đúng như tên wuan . Đồ_ăn đến thức_uống đều ngon giá hợp túi_tiền sanh viên . thich nhất mấy món kem và soda đồ ăn món nào cũng ok mì sản_phẩm ageti ly cream ngon thần_thánh của mềnh cơm gà chiên giòn cơm cuộn ăn đỡ ngán vì có sốt cà_chua gà bên trong hơi ít


In [88]:
multiple_output_post_processing(r, QA_M('restaurant').aspect_categories, label_list)

[['phong_cách đồ_ăn', 'tích_cực'],
 ['chất_lượng đồ_ăn', 'tích_cực'],
 ['dịch_vụ', 'tích_cực'],
 ['giá thức_ăn', 'tích_cực']]